# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_original = pd.read_csv("../output_data/cities1.csv")
city_data_original.head()

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,Lakes Entrance,-37.8811,147.9810,44.01,96,9,8.93,AU,1619469149
1,Lebu,-37.6167,-73.6500,59.00,73,2,12.57,CL,1619468889
2,Mataura,-46.1927,168.8643,39.99,89,100,1.01,NZ,1619468783
3,Punta Arenas,-53.1500,-70.9167,44.60,100,90,10.36,CL,1619468682
4,Moundsville,39.9203,-80.7431,66.99,31,1,3.04,US,1619469151


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
humidity_map = city_data_original[["Lat","Lng"]]

humidity_values = city_data_original["Humidity"]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(humidity_map, weights=humidity_values, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
'''
* Narrow down the DataFrame to find your ideal weather condition. For example:

  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

'''

"\n* Narrow down the DataFrame to find your ideal weather condition. For example:\n\n  * A max temperature lower than 80 degrees but higher than 70.\n\n  * Wind speed less than 10 mph.\n\n  * Zero cloudiness.\n\n  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.\n\n"

In [6]:
narrowed_df = city_data_original[((city_data_original['Max_Temp'] < 80) & (city_data_original['Max_Temp'] > 70)) &
                                (city_data_original['Wind_Speed'] < 10) &
                                 (city_data_original['Cloudiness'] == 0)]
narrowed_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
119,Linares,-35.8500,-71.6000,72.75,29,0,6.20,CL,1619469206
187,Karratha,-20.7377,116.8463,74.71,44,0,7.81,AU,1619468781
252,Bāzpur,29.1500,79.1167,79.18,12,0,4.45,IN,1619469288
261,Brufut,13.3814,-16.7518,73.40,73,0,9.22,GM,1619469292
287,Eldorado,-23.7869,-54.2836,70.34,46,0,7.20,BR,1619469309
563,Monte Patria,-30.6919,-70.9467,74.23,43,0,5.75,CL,1619469482


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels within 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = narrowed_df

params = {
    "radius": 5000,
    "key": g_key,
    "types": "lodging",
}


hotel_df['Lat']
hotel_df['Lat']

for index, row in hotel_df.iterrows():

    lat=row['Lat']
    lng=row['Lng']

    params['location'] = f"{lat}, {lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    name_address = requests.get(base_url, params=params)
    
    name_address = name_address.json()

    try:
        hotel_df.loc[index, "Hotel_Name"] = name_address['results'][0]['name']
    except:
        print('Missing field/result... skipping')

    coordinates = hotel_df[["Lat","Lng"]]

figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout,map_type='SATELLITE')

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)

# Add the layer to the map
fig.add_layer(heat_layer)
fig.add_layer(markers)
fig

C:\Users\17147\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\17147\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [8]:
hotel_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel_Name
119,Linares,-35.8500,-71.6000,72.75,29,0,6.20,CL,1619469206,Hotel Real
187,Karratha,-20.7377,116.8463,74.71,44,0,7.81,AU,1619468781,ibis Styles Karratha
252,Bāzpur,29.1500,79.1167,79.18,12,0,4.45,IN,1619469288,Hotel castle park bazpur
261,Brufut,13.3814,-16.7518,73.40,73,0,9.22,GM,1619469292,Sheraton Gambia Hotel Resort & Spa
287,Eldorado,-23.7869,-54.2836,70.34,46,0,7.20,BR,1619469309,Villa Verde Hotel
563,Monte Patria,-30.6919,-70.9467,74.23,43,0,5.75,CL,1619469482,Hotel Monte Cordillera


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map
fig1 = gmaps.figure(layout=figure_layout)

markers =gmaps.marker_layer(locations, info_box_content = hotel_info, display_info_box =True)
fig1.add_layer(markers)

# Display figure
fig1

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…